# Case Study 2 : Google Traffic Analysis

Steps :
- Make Meta2d and Cosinor analysis
- Plot Peaks
- Make Signal Analysis with pyBOAT
- Try using Rain

In [ ]:
#import
from rda_package import rda 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from CosinorPy import file_parser,cosinor

## Make Meta2d and Cosinor analysis

In [ ]:
filename= 'google_traffic_workdays.xlsx'
df = rda.cosinor_read(filename)
df['x']=df['x']*60
rda.export_csv(df,filename)
filename= f'{filename[:-5]}.csv'

In [ ]:
rda.cosinorpy(filename,sep='\t',period=24*60,n_components=3)
rda.cosinor1py(filename,sep='\t',period=24*60)

In [ ]:
print(filename)
rda.meta2d_format(filename=filename,sep='\t')
filename=f"Out/{filename[:-4]}/{filename[:-4]}.csv"
rda.meta2d(filename=filename)

## Plot Peaks

In [ ]:
#function
def plot_peaks(df,filename):
    names = df.test.unique()
    print(names)
    models_peak = [{"Path":f"Out/{filename[:-4]}/cosinorpyout/COSINORresult_{filename[:-4]}.csv","x_peak":"peaks","y_peak":"heights","label":"test","model":"cosinor"}]
    models_peak.append({"Path":f"Out/{filename[:-4]}/metaout/LSresult_{filename[:-4]}.csv","x_peak":"PeakSPD","y_peak":"PhaseShiftHeight","label":"CycID","model":"LS"})
    models_peak.append({"Path":f"Out/{filename[:-4]}/cosinorpyout/COSINOR1result_{filename[:-4]}.csv","x_peak":"acrophase[h]","y_peak":"amplitude","label":"test","model":"cosinor1"})
    models_peak.append({"Path":f"Out/{filename[:-4]}/cosinorpyout/COSINOR1result_{filename[:-4]}.csv","x_peak":"acrophase[h]","y_peak":"amplitude","label":"test","model":"cosinor1_MESOR"})
    models_peak.append({"Path":f"Out/{filename[:-4]}/metaout/JTKresult_{filename[:-4]}.csv","x_peak":"LAG","y_peak":"AMP","label":"CycID","model":"JTK"})
    models_peak.append({"Path":f"Out/{filename[:-4]}/metaout/JTKresult_{filename[:-4]}.csv","x_peak":"LAG","y_peak":"AMP","label":"CycID","model":"JTK_MESOR"})
    for name in names:
        ncols = 2
        nrows = 3
        fig, axes = plt.subplots(ncols = ncols, nrows = nrows, sharey=False)
        axes = axes.flatten()         
        fig.set_size_inches(10, 10)
        sns.set_style("white")
        flatui = ['#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525']
        mesor= df[ df['test']==name].y.mean()
        for ax,model in zip(axes,models_peak):
            print(name,model.get("model"))
            path=model.get("Path")
            df_peaks=pd.read_csv(path)
            dff=df[df['test']==name]
            df_peak=df_peaks[df_peaks[model.get("label")]==name]
            x_peak= str(df_peak[model.get("x_peak")].iloc[0]).strip('][').split(' ')
            y_peak=str(df_peak[model.get("y_peak")].iloc[0]).strip('][').split(' ')
            while("" in x_peak) :
                x_peak.remove("")
            while("" in y_peak) :
                y_peak.remove("")
            if(model.get("x_peak")=="LAG"):
                x_peak= [float(x)*60 if float(x)>0 else -float(x) for x in x_peak]
            else:
                x_peak= [float(x) if float(x)>0 else -float(x) for x in x_peak]
            print('x_peak:',x_peak,model.get("x_peak"))
            if model.get("model").split('_')[-1]=='MESOR':
                y_peak= [float(y)+mesor for y in y_peak]
            else:
                y_peak= [float(y) for y in y_peak]
            print('y_peak:',y_peak,model.get("y_peak"))
            sns.color_palette(flatui)
            sns.lineplot(data=dff,x='x',y='y',ax=ax,color = 'black')
            ax.set_xlabel(f'{model.get("x_peak")} ({model.get("model")})')
            ax.set_ylabel(f'{model.get("y_peak")} ({model.get("model")})')
            ax.plot(x_peak,y_peak,'ko')
        plt.suptitle(f"Peaks Plot")
        fig.subplots_adjust(top=0.95)
        plt.savefig(f"Out/{filename[:-4]}/Peaks_plot_{name}.png", bbox_inches="tight", facecolor='white')
        plt.show()

Make peaks of google traffic workdays

In [ ]:
filename='google_traffic_workdays.xlsx'
df = rda.cosinor_read(filename)
df['x']=df['x']*60
filename = f'{filename[:-5]}.csv'
plot_peaks(df,filename=filename)

Make peaks of google traffic

In [ ]:
filename='google_traffic.xlsx'
df = rda.cosinor_read(filename)
df['x']=df['x']*60
filename = f'{filename[:-5]}.csv'
plot_peaks(df,filename=filename)

## Make Signal Analysis with pyBOAT

In [ ]:
#import
from rda_package import rda 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from CosinorPy import file_parser,cosinor
filename='google_traffic.csv'
print(filename)
df = pd.read_csv(filename,sep='\t',index_col=0)
df_norep = df.index.to_series()
for i in range(0,1421,10):
    avg_filter = df.filter(regex=f'T{i}').mean(axis = 1)
    avg_filter.name = i
    df_norep = pd.merge(df_norep, avg_filter,left_index=True,right_index=True)
df_norep=df_norep.drop('gene',axis=1)

In [ ]:
df_ridge1 = rda.analysis(df_norep,filename=filename,time_unit_label='minutes')

## Try Rain

In [ ]:
#import
from rda_package import rda 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from CosinorPy import file_parser,cosinor
filename='google_traffic_workdays.csv'
print(filename)
df = pd.read_csv(filename,sep='\t',index_col=0)
df_norep = df.index.to_series()
for i in range(0,1421,10):
    avg_filter = df.filter(regex=f'T{i}').mean(axis = 1)
    avg_filter.name = i
    df_norep = pd.merge(df_norep, avg_filter,left_index=True,right_index=True)
df_norep=df_norep.drop('gene',axis=1)
df_norep.to_csv('norep_'+filename)

In [ ]:
rda.rain(filename='norep_'+filename,sample_rate=10,n_replicate=1,period=1440)